In [ ]:
# NOTES
# Correct! Here is a complete list of columns that hold alternate title data: Also known as, Arabic, Cantonese, Chinese, 
# French, Hangul, Hebrew, Hepburn, Japanese, Literally,  Mandarin, McCune–Reischauer, Original title, Polish,  
# Revised Romanization, Romanized, Russian, Simplified, Traditional, Yiddish

In [ ]:
import json
import pandas as pd
import numpy as np
import re

In [ ]:
file_dir = 'C:\\Users\kbead\Data Boot Camp\Modules\Module 8\Movies-ETL'
file_dir_rating = 'C:\\Users\kbead\Data Boot Camp\Modules\Module 8'

In [ ]:
with open(f'{file_dir}\wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
len(wiki_movies_raw)

In [ ]:
wiki_movies_raw[3600:3605]

In [ ]:
# read csv files using pandas
kaggle_metadata = pd.read_csv(f'{file_dir}\\movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir_rating}\\ratings.csv')

In [ ]:
# Skill drill 8.2.2 practice use of sample() method. Fora DataFrame called df, df.sample(n=5) will show five random rows
kaggle_metadata.sample(n=5)

In [ ]:
ratings.sample(n=10)

In [ ]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
wiki_movies_df.head()

In [ ]:
sorted(wiki_movies_df.columns.tolist())

In [ ]:
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie) 
                   and 'imdb_link' in movie 
                   and 'No. of episodes' not in movie]
len(wiki_movies)

In [ ]:
# wiki_movies_df = pd.DataFrame(wiki_movies)
# wiki_movies_df.head()

In [ ]:
def clean_movie(movie):
    movie = dict(movie) # <--create a non-destructive copy
    alt_titles = {}     # <--create empy dict to later populate
    
    # add list of alt keys to loop through
    for key in ['Arabic', 'Cantonese', 'Chinese', 'French', 'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally', 'Mandarin', 
                'McCune–Reischauer', 'Original title', 'Polish', 'Revised Romanization', 'Romanized', 'Russian', 'Simplified', 
                'Traditional', 'Yiddish']:
        
        if key in movie:                   # <--check if key is in movie object 
            alt_titles[key] = movie[key]   # <--add the alt titles to the dict
            movie.pop(key)                 # <--remove the key-value pair
            
    if len(alt_titles) > 0:              # <--make sure the dict has key-value pairs
        movie['alt_titles'] = alt_titles # <--add the dict to the movie object
          
    # Merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    # rename columns for easier readability
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    return movie 

In [ ]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
# use list comprehension to make a list of cleaned movies
clean_movies = [clean_movie(movie) for movie in wiki_movies]

# set the wiki_movies_df to be the DataFrame created from clean_movies
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

In [ ]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
print(len(wiki_movies_df))
wiki_movies_df.head()

In [ ]:
wiki_columns_to_keep = [column for column in wiki_movies_df.columns 
                     if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

In [ ]:
sorted(wiki_movies_df.columns.tolist())

In [ ]:
box_office = wiki_movies_df['Box office'].dropna()

In [ ]:
# def is_not_a_string(x):
#     return type(x) != str

#box_office[box_office.map(lambda x: type(x) != str)]

box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
box_office = box_office.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
box_office

In [ ]:
# regex to find box office values in the format $123.4 million/billion
# \$ searches for dollar sign at beginning
# \d+ searches for an arbitrary number of digits except zero amount
# \.? searches for a potential period
# \d* an arbitrary (but possibly zero) number of more digits
# \s* a space and possibly more than one
# [mb]illion looks for the word million or billion

form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

In [ ]:
# count the number of values that match the form $123.4 million/billion
box_office.str.contains(form_one, flags=re.IGNORECASE).sum()

In [ ]:
# regex to find box office values in the form $1,234,567
# \$ search for dollar sign at beginning
# \d{1,3} search for a combination of 1 to 3 digits
# (?:,\d{3})+ searches for combination of up 3 digits after the comma for as many times as it occurs.
# the ?: is stated to not be quite necessary here but it is used to eliminate a warning message from Jupyter Notebook

form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

In [ ]:
box_office.str.contains(form_two, flags=re.IGNORECASE).sum()

In [ ]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)

In [ ]:
# this will throw an error!
box_office[~matches_form_one & ~matches_form_two]

In [ ]:
box_office.str.extract(f'({form_one}|{form_two})')

In [ ]:
def parse_dollars(s):
    # if s is not a string, return NaN
    if type(s) != str:
        return np.nan

    # if input is of the form $###.# million
    if re.match(r'\$\s*\d+\.?\d*\s*[mb]illi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value

    # if input is of the form $###.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*[mb]illi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
        s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion
        value = float(s) * 10**9

        # return value
        return value

    # if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
        s = re.sub('\$|,','', s)

        # convert to float
        value = float(s)

        # return value
        return value

    # otherwise, return NaN
    else:
        return np.nan

In [ ]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df['box_office']

In [ ]:
wiki_movies_df.drop('Box office', axis=1, inplace=True)

In [ ]:
# Create a budget variable
budget = wiki_movies_df['Budget'].dropna()

In [ ]:
# Convert any lists to strings
budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
# Remove any values between a dollar sign and a hyphen
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)

In [ ]:
# Use form_one and form_two that were created earlier and find string values using the regex code
matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
budget[~matches_form_one & ~matches_form_two]

In [ ]:
# Remove the citation references (numbers inside brackets)
budget = budget.str.replace(r'\[\d+\]\s*', '')
budget[~matches_form_one & ~matches_form_two]

In [ ]:
# Copied the code from parsing the box office values and changed 'box_office' variables to 'budget'
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
wiki_movies_df['budget']

In [ ]:
# Drop the original Budget column
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [118]:
# Create a variable that holds non-null values of Release date in DataFrame, converting lists to strings
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [121]:
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s*\d{1,2},\s*\d{4}'
date_form_two = r'\d{4}.\d{2}.\d{2}'
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'

In [122]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

,0
0,"July 11, 1990"
1,"May 17, 1990"
2,"August 10, 1990"
3,"December 25, 1990"
4,"December 19, 1990"
...,...
7071,"December 25, 2018"
7072,"December 11, 2018"
7073,"November 8, 2018"
7074,"August 31, 2018"


In [123]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [125]:
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ''.join(x) if type(x) == list else x)

In [126]:
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE).sum()

6528

In [127]:
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]

9                                                 102 min
26                                                 93 min
28                                                32 min.
34                                                101 min
35                                                 97 min
                              ...                        
6500         114 minutes[1]120 minutes (extended edition)
6643                                             104 mins
6709      90 minutes(theatrical)[1]91 minutes(unrated)[2]
7057    108 minutes(Original cut)98 minutes(UK cut)93 ...
7075                Variable; 90 minutes for default path
Name: Running time, Length: 366, dtype: object

In [128]:
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()

6877

In [130]:
running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE) != True]

668                      UK:84 min (DVD version)US:86 min
727                         78-102 min (depending on cut)
840                           Varies (79[3]–84[1]minutes)
1347                                                25:03
1443    United States:77 minutesArgentina:94 minutesGe...
1499                                            1hr 35min
1551                                               varies
1774                    Netherlands:96 min, Canada:95 min
1777                                       approx. 14 min
2273                                           1 h 43 min
2993                                               1h 48m
3925                                              4 hours
4425    US domestic version:86 minutesOriginal version...
4967     Theatrical cut:97 minutesUnrated cut:107 minutes
5424                          115[1]/123[2]/128[3]minutes
5447                                    1 hour 32 minutes
7075                Variable; 90 minutes for default path
Name: Running 

In [131]:
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [132]:
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [133]:
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [134]:
wiki_movies_df.drop('Running time', axis=1, inplace=True)